### Raw Hate Speech (HS) Data - Parsing, Preprocessing and Text Analysis

# Data Frequency Analysis - as unigram (1-gram) analysis

The scripts were created and published by Merja Kreivi-Kauppinen, and
are part of research work carried in University of Oulu in 2020-2023.

The study is reported on (Master’s Thesis) research paper - 
Merja Kreivi-Kauppinen (2024) Hate Speech Detection of Dialectal, Granular and Urban Finnish. 
University of Oulu, Degree Programme in Computer Science and Engineering. Master’s Thesis.

### Import python packages and libraries

In [1]:
# import python packages and libraries
import sys, re, os, openpyxl
import json
import time, datetime, random, string

import numpy as np
import pandas as pd

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
from sklearn.feature_extraction.text import CountVectorizer

# import NLTK methods and libraries
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import casual_tokenize, word_tokenize

import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from pylab import rcParams

# set seaborn figures
%matplotlib inline
rcParams['figure.figsize'] = 8, 4
sns.set(style='darkgrid', palette='muted', font_scale=1.0)

The scikit-learn version is 1.1.2.


## Check the list of stopwords - Merias list of stopwords in Finnish

In [2]:
# Meria's list for stopwords
# import data from txt file as words

cwd = os.getcwd()
folder = '\\data\\'
file = 'finnish_stopwords_Merias_list.txt'
source = cwd + folder + file
FinnishStopwords = set(open(source, "r", encoding='utf-8').read().split())

print(FinnishStopwords)
#FinnishStopwords

{'kunhan', 'näistä', 'tuota', 'ollut', 'niitä', 'tälle', 'noiksi', 'sinusta', 'keihin', 'siitä', 'vaikka', 'häneltä', 'elikkä', 'kunnes', 'näihin', 'ole', 'meidät', 'heidät', 'johon', 'minkä', 'millä', 'ne', 'minua', 'näillä', 'sinut', 'kuten', 'jona', 'kenestä', 'sitten', 'teillä', 'tältä', 'jota', 'josta', 'kuin', 'minulla', 'noin', 'meistä', 'mikäli', 'silti', 'jne', 'joille', 'jolta', 'joko', 'keillä', 'kuka', 'näille', 'täksi', 'ketkä', 'joita', 'tuohon', 'siltä', 'sinua', 'niinkuin', 'keinä', 'minun', 'joina', 'hänelle', 'heihin', 'tätä', 'keitä', 'meiltä', 'teidän', 'keneltä', 'kenessä', 'joihin', 'taikka', 'ett', 'noiden', 'sillä', 'he', 'hänestä', 'joilta', 'ketä', 'siksi', 'mille', 'miltä', 'niiltä', 'niistä', 'olisitte', 'keneen', 'nyt', 'te', 'teille', 'niiden', 'sinulle', 'hänet', 'teissä', 'myös', 'nuo', 'tuoksi', 'noina', 'joiden', 'minusta', 'tänä', 'se', 'missä', 'meille', 'minussa', 'keille', 'olin', 'mikä', 'jos', 'ovat', 'sille', 'noille', 'on', 'meitä', 'niihin', '

## Check the list of sentiment words in Finnish

In [3]:
# Download sentiment words - import data from txt file as words
cwd = os.getcwd()
folder = '\\data\\'
file = 'AFINN_FIN_sentiment_words.txt'
source = cwd + folder + file
#print(source)

data = []

def delete_new_line(value):
    return ''.join(value.splitlines())

with open(source, "r", encoding='utf-8') as file:
#with open(source, "r") as file:
    lines = file.readlines()
    for line in lines:
        line_items = line.split('\t')        
        if (len(line_items) > 1):
            line_items[1] = delete_new_line(line_items[1])
            data.append(line_items)

# pass the chunks in pd.DataFrame and specify the column names
df_sentwords = pd.DataFrame(data, columns=["word", "value"])
df_sentwords

,word,value
0,ööö,-1
1,öö,-1
2,öykkäröinti,-3
3,öykkäröidä,-3
4,öykkärimäisyys,-3
...,...,...
11667,aaltoilla,-1
11668,aalto,-1
11669,aallot,-1
11670,1v1,-5


In [4]:
finnish_sentiment_words = set(df_sentwords['word'])
#finnish_sentiment_words
print(finnish_sentiment_words)

{'tarvi', 'siunattu', 'posttraumaattiset', 'laukaista', 'kriisiytyä', 'huonous', 'ratkaisuja', 'hinttari', 'poikkeava', 'arvo', 'yököttävä', 'luukku', 'karkotettu', 'hyväksyttävä', 'kehar', 'anella', 'häväis', 'riitelevät', 'räävittömät', 'rahvas', 'monopolisoida', 'ailahte', 'ahne', 'tervehdyttävä', 'runsaus', 'alamäki', 'fags', 'epäuskot', 'ätäkin', 'haittapuoli', 'uskovai', 'säästeliäs', 'täynnä', 'kriminaali', 'pompata', 'epämukava', 'ruuhkautua', 'sadunomainen', 'avioero', 'rikastua', 'somistuttaa', 'piilotettu', 'hompsa', 'positiivisesti', 'uhmaikä', 'pieksetään', 'tappoi', 'pidättele', 'paatoksellinen', 'maailmanmestari', 'rapaa', 'hulluryssät', 'gaye', 'asiantuntija', 'biseksu', 'epäilys', 'ripuloida', 'wow', 'nälissään', 'hintava', 'pässi', 'koulukiusaaminen', 'sairaus', 'äreästi', 'ylistävä', 'bottari', 'rakkaudet', 'voimassa', 'provo', 'silmä', 'aavikoitua', 'vastaan', 'hienoksi', 'epätäydellisesti', 'ammattitaitoinen', 'etsiä', 'rikkeet', 'ennakkosuosikki', 'varoittava', 't

## Download Spacy + Voikko Method

### spacy-fi asennus ohjeet

Asenna spacy 

    - pip install -U pip setuptools wheel
    
    - pip install -U spacy

Install the English language model by running:

    - python -m spacy download en_core_web_sm

Install the Finnish language model by running:

    - pip install spacy_fi_experimental_web_md

In [5]:
# Import and check Spacy
import os
import spacy
print(f"Spacy Version: {spacy.__version__}")

Spacy Version: 3.5.0


In [6]:
# Test large Finnish model

# Load large Finnish model
FinSpacyLarge = spacy.load("fi_core_news_lg")

# Test the model
doc = FinSpacyLarge('Terveyden ja hyvinvoinnin laitos (THL) johtaa hanketta. Ilmassa on kevättä. Rännit tippuvat ja katuojat juoksevat.')
for t in doc:
    print(f'{t.lemma_}\t{t.pos_}')

terveys	NOUN
ja	CCONJ
hyvinvointi	NOUN
laitos	NOUN
(	PUNCT
THL	NOUN
)	PUNCT
johtaa	VERB
hanke	NOUN
.	PUNCT
Ilmassa	NOUN
olla	AUX
kevät	NOUN
.	PUNCT
Rännit	NOUN
tippua	VERB
ja	CCONJ
katuoja	NOUN
juoksevat	VERB
.	PUNCT


### Voikko asennus ohjeet

Voikko asennusohjeet pythonille windowsilla

    https://voikko.puimula.org/python.html

Asenna libvoikko anacondalle haluamaasi kansioon komennolla

    - pip3 install libvoikko

Rakenna Voikko kansio esimerkiksi suoraan C: kansioon

C:/Voikko/

    - libvoikko-1.dll
    
    - libvoikko.dll.a
    
    - libvoikko-4.3+win1.tar.gz
    
    - 5/(v5 VFST sanasto kansiot auki zipattuna)

Voikko sanastot kansiot löytyvät osoitteesta

    https://www.puimula.org/htp/testing/voikko-snapshot-v5/

In [7]:
from libvoikko import Voikko

In [8]:
# set up Voikko search path for Voikko help
Voikko.setLibrarySearchPath("c:/Voikko")
help(Voikko)

Help on class Voikko in module libvoikko:

class Voikko(builtins.object)
 |  Voikko(language, path=None)
 |  
 |  Represents an instance of Voikko. The instance has state, such as
 |  settings related to spell checking and hyphenation, and methods for performing
 |  various natural language analysis operations. One instance should not be
 |  used simultaneously from multiple threads.
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __init__(self, language, path=None)
 |      Creates a new Voikko instance with the following optional parameters:
 |      language  BCP 47 language tag to be used.
 |      path      Extra path that will be checked first when looking for linguistic
 |                resources.
 |  
 |  analyze(self, word)
 |      Analyze the morphology of given word and return the list of
 |      analysis results. The results are represented as maps having property
 |      names as keys and property values as values.
 |  
 |  attributeValues(self, attributeName)

In [9]:
# get Voikko directory list
print("\nVoikko directory list:  ", Voikko.listDicts("C:/Voikko"))


Voikko directory list:   [<fi,,standard,suomi (perussanasto)>, <fi,,dialects,suomi (murteellisten, vanhojen ja harvinaisten sanojen sanasto)>, <fi,,education,suomi (kasvatustieteen sanasto)>, <fi,,medicine,suomi (matematiikan, fysiikan, kemian, biologian, maantieteen, geologian ja lääketieteen sanastot)>, <fi,,morpho,suomi, erittäin laaja sanasto (mukana myös morfologisessa analyysissä tarvittava lisäinformaatio)>, <fi,,morphoid,suomi, erittäin laaja sanasto (mukana myös morfologisessa analyysissä tarvittava lisäinformaatio ja linkitys Joukahaiseen)>, <fi,,po,suomi (po-tiedostojen oikolukusanasto)>, <fi,,science,suomi (matematiikan, fysiikan, kemian, biologian, maantieteen, geologian, lääketieteen, kasvatustieteen ja tietotekniikan sanastot)>]


In [10]:
# get Voikko version
print("Voikko version:  ", Voikko.getVersion())

Voikko version:   4.3


In [11]:
print("\nCurrent Working Directory " , os.getcwd())


Current Working Directory  C:\Users\merja\Desktop\HS_research_project\DataAnalysis


In [12]:
# Test experimental Finnish Spacy model with Spacy

# Load large Finnish model
FinSpacyExp = spacy.load("spacy_fi_experimental_web_md")

# Test the model
doc = FinSpacyExp('Terveyden ja hyvinvoinnin laitos (THL) johtaa hanketta. Ilmassa on kevättä. Rännit tippuvat ja katuojat juoksevat.')
for t in doc:
    print(f'{t.lemma_}\t{t.pos_}')

terveys	NOUN
ja	CCONJ
hyvinvointi	NOUN
laitos	NOUN
(	PUNCT
THL	PROPN
)	PUNCT
johtaa	VERB
hanke	NOUN
.	PUNCT
ilma	NOUN
olla	AUX
kevät	NOUN
.	PUNCT
ränni	NOUN
tippua	VERB
ja	CCONJ
katuoja	NOUN
juosta	VERB
.	PUNCT


# Download HS Dataset for Analysis

## Download manually annotated collection of all HS data


### Test tokenization with NLTK 'casual_tokenization' and lemmatization with 'Experimental' Fin Spacy

In [13]:
# import data from xlsx file to pd dataFolder
cwd = os.getcwd()
folder = '\\data\\'
#csv_file = 'HSTrainData.xlsx'
csv_file = 'Manually_Annotated_Collection_ALL_FINAL_22023Merja.xlsx'
csv_source = cwd + folder + csv_file
#print(csv_source)
df = pd.read_excel(csv_source)
df

# Change text samples to string
df["stringSample"] = df[str("sample")]
df["stringCorrect"] = df[str("correct_finnish")]

# change sentiment values
df['sentiment'] = df['sentiment'].replace('neutral', 0)
df['sentiment'] = df['sentiment'].replace('positive', 1)
df['sentiment'] = df['sentiment'].replace('negative', 2)

# change sentiment values
df['HSbinary'] = df['HSbinary'].replace(False, 0)
df['HSbinary'] = df['HSbinary'].replace(True, 1)

df.head(50)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu


In [14]:
# Tokenization of raw samples with NLTK casual tokenization
df['sampleTokens'] = df['stringSample'].apply(lambda row: casual_tokenize(row))

# Tokenization of corrected samples with NLTK casual tokenization
df['correctTokens'] = df['stringCorrect'].apply(lambda row: casual_tokenize(row))

#df.head(10)

# Lemmatization of raw samples
df['sampleLemmas'] = df['stringSample'].apply(lambda row: " ".join([token.lemma_ for token in FinSpacyExp(row)]))

# Lemmatization of corrected samples
df['correctLemmas'] = df['stringCorrect'].apply(lambda row: " ".join([token.lemma_ for token in FinSpacyExp(row)]))
df.head(10)


,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,"[-, Ajatus, siitä, että, kaikki, henkilön, tie...","[-, Ajatus, siitä, että, kaikki, henkilön, tie...",- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]",--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]",-=>Widdu joo<= -,-= > vittu joo < = -
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...","[-, Ei, ,, mutta, olen, joutunut, elämään, kat...","[Ei, ,, mutta, olen, joutunut, elämään, katsom...","- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,"[-, ei, viddu, -]","[ei, vittu]",- ei viddu -,ei vittu
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,"[-, Huvittunut, -]",[huvittunut],- huvittunut -,huvittua
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...","[-, Hyvä, että, JOKU, näkee, uuden, ,, tyhmän,...","[Hyvä, että, JOKU, näkee, uuden, ,, tyhmän, ve...","- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,"[-, Kukaan, luterilainen, ei, korota, itseään,...","[Kukaan, luterilainen, ei, korota, itseään, hä...",- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,"[-, kuten, on, sovittu, -]","[kuten, on, sovittu]",- kuten olla sopia -,kuten olla sopia


# Token frequency analysis - as unigram analysis (1-gram analysis)

## Token frequency analysis for 'raw' HS samples

In [15]:
df_HS = df[df['HSbinary'] >= 1]
df_HS

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]",--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]",-=>Widdu joo<= -,-= > vittu joo < = -
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,"[-, ei, viddu, -]","[ei, vittu]",- ei viddu -,ei vittu
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...","[-, Hyvä, että, JOKU, näkee, uuden, ,, tyhmän,...","[Hyvä, että, JOKU, näkee, uuden, ,, tyhmän, ve...","- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ..."
11,12,- Tarpeeksi vihainen mies tappaa nyrkillä kius...,2,-5,1,3,GROUP,FAMILY SCHOOL FRIENDS,"INCITEMENT, VIOLENCE","ANGER HATE, AGGRESSIVE VIOLENT",- Tarpeeksi vihainen mies tappaa nyrkillä kius...,Tarpeeksi vihainen mies tappaa nyrkillä kiusaa...,Meria,- Tarpeeksi vihainen mies tappaa nyrkillä kius...,Tarpeeksi vihainen mies tappaa nyrkillä kiusaa...,"[-, Tarpeeksi, vihainen, mies, tappaa, nyrkill...","[Tarpeeksi, vihainen, mies, tappaa, nyrkillä, ...",- tarpeeksi vihainen mies tappaa nyrkki kiusaa...,tarpeeksi vihainen mies tappaa nyrkki kiusaaja...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6611,6612,ööliä vittu,2,-4,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",CONTEMPT DISRESPECT,ööliä vittu,olutta vittu,Meria,ööliä vittu,olutta vittu,"[ööliä, vittu]","[olutta, vittu]",ööliä vittu,olut vittu
6612,6613,öö-luokkalaiset.vittuun,2,-5,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING",SARCASTIC,öö-luokkalaiset vittuun,öö-luokkalaiset vittuun,Meria,öö-luokkalaiset.vittuun,öö-luokkalaiset vittuun,"[öö-luokkalaiset, ., vittuun]","[öö-luokkalaiset, vittuun]",öö-luokkalaiset.vittuun,öö-luokkalaiset vittu
6613,6614,öö-vittu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",DISGUST,öö -vittu,öö vittu,Meria,öö-vittu,öö vittu,[öö-vittu],"[öö, vittu]",öö-vittu,öö vittu
6615,6616,Ööö mitä vittua??,2,-4,1,2,NONE,OPINION,"JOKE SARCASM, SWEARING",SURPRISE,Ööö mitä vittua??,"Öö, mitä vittua??",Meria,Ööö mitä vittua??,"Öö, mitä vittua??","[Ööö, mitä, vittua, ?, ?]","[Öö, ,, mitä, vittua, ?, ?]",Ööö mitä vittu ? ?,"Öö , mitä vittu ? ?"


In [16]:
# only tokenized !!!! - shows tokens with upper characters
all_sample_tokens = []

# collect all HS samples (!) as tokens to one dict
for sample in df_HS['sampleTokens']:
    for token in sample:
        all_sample_tokens.append(token)

print("Len of all_sample_tokens:  ", len(all_sample_tokens))
print("\nAll tokens:  ", all_sample_tokens)

Len of all_sample_tokens:   28110

All tokens:   ['-', '-', '´', '973', '´', '¤', '-', '.', 'ttu', '-', '=', '>', 'Widdu', 'joo', '<', '=', '-', '-', 'ei', 'viddu', '-', '-', 'Hyvä', 'että', 'JOKU', 'näkee', 'uuden', ',', 'tyhmän', 'veroperusteen', 'jossa', 'tuijotetaan', 'VAIN', 'CO', 'arvoon', 'ja', 'diesel', 'paskaa', 'noella', 'ilman', 'ja', 'keuhkot', '.', '-', 'Tarpeeksi', 'vihainen', 'mies', 'tappaa', 'nyrkillä', 'kiusaajansa', ',', 'kun', 'mitta', 'tulee', 'täyteen', '...', '-', 'vittutieteen', 'tohtori', '-', '”', 'Käristy', 'helvetin', 'tulessa', '”', '!', 'Jos', 'mies', 'vastaavasti', 'pukee', 'itsensä', 'niin', ',', 'että', 'pallit', 'tai', 'mulkku', 'näkyy', ',', 'niin', 'johan', 'sitä', 'kauhistelee', 'naurismaan', 'aidatkin', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noo

In [17]:
# Use counter to count the frequency of words / tokens / lemmas
from collections import Counter

# Count the token unigrams
N1gram = Counter(all_sample_tokens)
N1gram

# N2gram, N3gram and N4gram analysis with different process !!!!!!!!!!!!!!!!!!!
# NLTK ngrams enable to group words in unigrams, twograms, trigrams, fourgrams etc.
# from nltk import ngrams

Counter({'-': 167,
         '´': 10,
         '973': 1,
         '¤': 7,
         '.': 1000,
         'ttu': 54,
         '=': 22,
         '>': 15,
         'Widdu': 11,
         'joo': 39,
         '<': 1,
         'ei': 334,
         'viddu': 31,
         'Hyvä': 7,
         'että': 255,
         'JOKU': 2,
         'näkee': 2,
         'uuden': 5,
         ',': 877,
         'tyhmän': 1,
         'veroperusteen': 1,
         'jossa': 22,
         'tuijotetaan': 1,
         'VAIN': 2,
         'CO': 1,
         'arvoon': 1,
         'ja': 573,
         'diesel': 1,
         'paskaa': 61,
         'noella': 1,
         'ilman': 11,
         'keuhkot': 1,
         'Tarpeeksi': 1,
         'vihainen': 1,
         'mies': 15,
         'tappaa': 12,
         'nyrkillä': 2,
         'kiusaajansa': 1,
         'kun': 153,
         'mitta': 2,
         'tulee': 19,
         'täyteen': 2,
         '...': 309,
         'vittutieteen': 1,
         'tohtori': 1,
         '”': 15,
         'Käri

In [18]:
# Create data frame for HS tokens in 'raw' samples category
freq_N1 = pd.DataFrame(N1gram.items(),columns=['token','frequency']).sort_values(by='frequency',ascending=False)
freq_N1.head(30)

,token,frequency
4,.,1000
18,",",877
75,vittu,863
49,!,862
26,ja,573
65,on,452
316,Vittu,361
394,mikä,336
11,ei,334
42,...,309


## Token frequency analysis for manually 'corrected' HS samples

In [19]:
# only tokenized !!!! - shows tokens with upper characters
all_correct_tokens = []

# collect all HS samples (!) as tokens to one dict
for sample in df_HS['correctTokens']:
    for token in sample:
        all_correct_tokens.append(token)

print("Len of all_correct_tokens:  ", len(all_correct_tokens))
print("All tokens:  ", all_correct_tokens)

Len of all_correct_tokens:   30414
All tokens:   ['973', 'vittu', '-', '=', '>', 'vittu', 'joo', '<', '=', '-', 'ei', 'vittu', 'Hyvä', 'että', 'JOKU', 'näkee', 'uuden', ',', 'tyhmän', 'veroperusteen', ',', 'jossa', 'tuijotetaan', 'VAIN', 'CO', 'arvoon', 'ja', 'diesel', 'paskaa', 'noella', 'ilman', 'ja', 'keuhkot', '.', 'Tarpeeksi', 'vihainen', 'mies', 'tappaa', 'nyrkillä', 'kiusaajansa', ',', 'kun', 'mitta', 'tulee', 'täyteen', '...', 'vittutieteen', 'tohtori', '”', 'Käristy', 'helvetin', 'tulessa', '”', '!', 'Jos', 'mies', 'vastaavasti', 'pukee', 'itsensä', 'niin', ',', 'että', 'pallit', 'tai', 'mulkku', 'näkyy', ',', 'niin', 'johan', 'sitä', 'kauhistelee', 'naurismaan', 'aidatkin', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', '!', '!', '!', '!', 'mopar', 'on', 'paska',

In [20]:
# Use counter to count the frequency of words / tokens / lemmas
from collections import Counter

# Count the token unigrams
N1gram = Counter(all_correct_tokens)
N1gram

# N2gram, N3gram and N4gram analysis with different process !!!!!!!!!!!!!!!!!!!
# NLTK ngrams enable to group words in unigrams, twograms, trigrams, fourgrams etc.
# from nltk import ngrams

Counter({'973': 1,
         'vittu': 1752,
         '-': 109,
         '=': 19,
         '>': 2,
         'joo': 56,
         '<': 2,
         'ei': 403,
         'Hyvä': 7,
         'että': 302,
         'JOKU': 2,
         'näkee': 2,
         'uuden': 5,
         ',': 1086,
         'tyhmän': 1,
         'veroperusteen': 1,
         'jossa': 23,
         'tuijotetaan': 1,
         'VAIN': 2,
         'CO': 1,
         'arvoon': 1,
         'ja': 591,
         'diesel': 1,
         'paskaa': 87,
         'noella': 1,
         'ilman': 11,
         'keuhkot': 1,
         '.': 1045,
         'Tarpeeksi': 1,
         'vihainen': 1,
         'mies': 16,
         'tappaa': 46,
         'nyrkillä': 2,
         'kiusaajansa': 1,
         'kun': 209,
         'mitta': 2,
         'tulee': 20,
         'täyteen': 2,
         '...': 195,
         'vittutieteen': 1,
         'tohtori': 1,
         '”': 11,
         'Käristy': 1,
         'helvetin': 12,
         'tulessa': 1,
         '!': 848,

In [21]:
# create data frame for HS tokens
freq_N1 = pd.DataFrame(N1gram.items(),columns=['token','frequency']).sort_values(by='frequency',ascending=False)
freq_N1.head(30)

,token,frequency
1,vittu,1752
13,",",1086
27,.,1045
45,!,848
21,ja,591
61,on,491
7,ei,403
328,mikä,395
9,että,302
313,mitä,280


## Unigram analysis for cleaned tokenized samples

### Lemma Frequency Analysis for all data in 'raw' and 'correct' samples

### Preprocessed by basic cleaning process
- leave only upper and lower characters
- tokenization
- lower
- leave only alphabet
- remove Finnish stopwords (Merja's list of stopwords)
- remove Finnish sentiment words (Merja's list of sentiment / emotion words)

In [22]:
# concatenate all tokens together as one list
all_tokens = all_sample_tokens + all_correct_tokens
print("Len of all_tokens:  ", len(all_tokens))
print("All tokens:  ", all_tokens)

Len of all_tokens:   58524
All tokens:   ['-', '-', '´', '973', '´', '¤', '-', '.', 'ttu', '-', '=', '>', 'Widdu', 'joo', '<', '=', '-', '-', 'ei', 'viddu', '-', '-', 'Hyvä', 'että', 'JOKU', 'näkee', 'uuden', ',', 'tyhmän', 'veroperusteen', 'jossa', 'tuijotetaan', 'VAIN', 'CO', 'arvoon', 'ja', 'diesel', 'paskaa', 'noella', 'ilman', 'ja', 'keuhkot', '.', '-', 'Tarpeeksi', 'vihainen', 'mies', 'tappaa', 'nyrkillä', 'kiusaajansa', ',', 'kun', 'mitta', 'tulee', 'täyteen', '...', '-', 'vittutieteen', 'tohtori', '-', '”', 'Käristy', 'helvetin', 'tulessa', '”', '!', 'Jos', 'mies', 'vastaavasti', 'pukee', 'itsensä', 'niin', ',', 'että', 'pallit', 'tai', 'mulkku', 'näkyy', ',', 'niin', 'johan', 'sitä', 'kauhistelee', 'naurismaan', 'aidatkin', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', '!', '!', '!', '!', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'h

In [23]:
#----------- lower characters -------------------------#
#----------- clean special chars and numbers ----------#

cleaned_tokens = []

# cleaning - remove all non alphabets
regex = re.compile('[\W]+')

for sample in all_tokens:
    sample = casual_tokenize(sample)
    tokens = [token.lower() for token in sample]
    tokens = [regex.sub(' ', token) for token in tokens]
    tokens = [token for token in tokens if token.isalpha()]
    clean_tokens = ' '.join(tokens)
    cleaned_tokens.append(clean_tokens)

# ------------ word tokenization -----------
list_of_all_tokens =" ".join(cleaned_tokens)
list_of_all_tokens = word_tokenize(list_of_all_tokens)

print('\nLen of tokenized samples:   ', len(list_of_all_tokens))
print('\nList of tokenized samples:   ', list_of_all_tokens)


Len of tokenized samples:    48179

List of tokenized samples:    ['ttu', 'widdu', 'joo', 'ei', 'viddu', 'hyvä', 'että', 'joku', 'näkee', 'uuden', 'tyhmän', 'veroperusteen', 'jossa', 'tuijotetaan', 'vain', 'co', 'arvoon', 'ja', 'diesel', 'paskaa', 'noella', 'ilman', 'ja', 'keuhkot', 'tarpeeksi', 'vihainen', 'mies', 'tappaa', 'nyrkillä', 'kiusaajansa', 'kun', 'mitta', 'tulee', 'täyteen', 'vittutieteen', 'tohtori', 'käristy', 'helvetin', 'tulessa', 'jos', 'mies', 'vastaavasti', 'pukee', 'itsensä', 'niin', 'että', 'pallit', 'tai', 'mulkku', 'näkyy', 'niin', 'johan', 'sitä', 'kauhistelee', 'naurismaan', 'aidatkin', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', 'mopar', 'on', 'paska', 'paikka', 'jossa', 'on', 'noobit', 'homot', 'mopar', 'on', 'paska', 'paikka', 'jossa

In [24]:
# ------------ remove stopwords ------------
stopwordsRemoved = []

#stopwordsFinnish = set(stopwords.words("finnish"))
stopwordsFinnish = FinnishStopwords
stopwordsRemoved = [word for word in list_of_all_tokens if word not in stopwordsFinnish]

stopwordsRemoved = []

for word in list_of_all_tokens:
    if word not in stopwordsFinnish:
        stopwordsRemoved.append(word)

print('\nLen of stopwordsRemoved:   ', len(stopwordsRemoved))
print('\nStopwords removed of all words list\n')
print(stopwordsRemoved)


Len of stopwordsRemoved:    37759

Stopwords removed of all words list

['ttu', 'widdu', 'joo', 'ei', 'viddu', 'hyvä', 'näkee', 'uuden', 'tyhmän', 'veroperusteen', 'tuijotetaan', 'vain', 'co', 'arvoon', 'diesel', 'paskaa', 'noella', 'ilman', 'keuhkot', 'tarpeeksi', 'vihainen', 'mies', 'tappaa', 'nyrkillä', 'kiusaajansa', 'mitta', 'tulee', 'täyteen', 'vittutieteen', 'tohtori', 'käristy', 'helvetin', 'tulessa', 'mies', 'vastaavasti', 'pukee', 'itsensä', 'pallit', 'mulkku', 'näkyy', 'johan', 'kauhistelee', 'naurismaan', 'aidatkin', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'homot', 'mopar', 'paska', 'paikka', 'noobit', 'ho

In [25]:
# ------------ remove Finnish sentiment words ------------

sentimentsRemoved = []

sentimentsFinnish = finnish_sentiment_words
sentimentsRemoved = [word for word in stopwordsRemoved if word not in sentimentsFinnish]

sentimentsRemoved = []

for word in stopwordsRemoved:
    if word not in sentimentsFinnish:
        sentimentsRemoved.append(word)

print('\nLen of stopwordsRemoved:   ', len(sentimentsRemoved))
print('\nSentiment words removed of all words list\n')
print(sentimentsRemoved)


Len of stopwordsRemoved:    19203

Sentiment words removed of all words list

['ttu', 'näkee', 'uuden', 'tyhmän', 'veroperusteen', 'tuijotetaan', 'co', 'arvoon', 'diesel', 'noella', 'keuhkot', 'kiusaajansa', 'mitta', 'tulee', 'täyteen', 'vittutieteen', 'tohtori', 'käristy', 'tulessa', 'vastaavasti', 'pukee', 'pallit', 'näkyy', 'johan', 'kauhistelee', 'naurismaan', 'aidatkin', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'mopar', 'homot', 'isäsi', 'kyrpää', 'jotkut', 'jaksavat', 'jostain', 'jotkut', 'jaksavat', 'viljellä', 'sitähän', 'minäkin', 'kokoaika', 'sanonut', 'pienimuotoisessa', 'lakossa', 'olo', 'lasta', 'uhrinsa', 'abortööri', 'lasta', 'seksuaalisesti', 'saadakseen', 'vastuuta', 'saa', 'pornoa', 'katse

In [26]:
# NLTK ngrams enable to group words in twograms, trigrams, fourgrams etc.
from nltk import ngrams

# Use counter to count the words / lemmas
from collections import Counter

# Count the token unigrams
N1gram = Counter(sentimentsRemoved)
N1gram

# N2gram, N3gram and N4gram analysis with different process !!!!!!!!!!!!!!!!!!!
# NLTK ngrams enable to group words in unigrams, twograms, trigrams, fourgrams etc.
# from nltk import ngrams

Counter({'ttu': 56,
         'näkee': 6,
         'uuden': 10,
         'tyhmän': 2,
         'veroperusteen': 2,
         'tuijotetaan': 2,
         'co': 2,
         'arvoon': 2,
         'diesel': 2,
         'noella': 2,
         'keuhkot': 2,
         'kiusaajansa': 2,
         'mitta': 6,
         'tulee': 42,
         'täyteen': 4,
         'vittutieteen': 2,
         'tohtori': 2,
         'käristy': 2,
         'tulessa': 2,
         'vastaavasti': 4,
         'pukee': 2,
         'pallit': 3,
         'näkyy': 4,
         'johan': 2,
         'kauhistelee': 2,
         'naurismaan': 2,
         'aidatkin': 2,
         'mopar': 38,
         'homot': 53,
         'isäsi': 2,
         'kyrpää': 27,
         'jotkut': 16,
         'jaksavat': 4,
         'jostain': 8,
         'viljellä': 2,
         'sitähän': 2,
         'minäkin': 8,
         'kokoaika': 2,
         'sanonut': 5,
         'pienimuotoisessa': 2,
         'lakossa': 2,
         'olo': 4,
         'lasta': 10,
  

In [27]:
# create data frame for token unigrams
df_results = pd.DataFrame(N1gram.items(),columns=['token','frequency']).sort_values(by='frequency',ascending=False)
df_results.head(30)

,token,frequency
873,en,132
856,mitään,99
258,et,76
7039,matu,72
1091,eikä,61
48,saa,59
0,ttu,56
1043,jotain,54
28,homot,53
812,vittuile,50


In [28]:
# load dataframe to csv at at output folder
def load_N1grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_tokens_N1gram_CSV_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to txt at output folder
def load_N1grams_toTXT(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    file = 'fin_tokens_N1gram_TXT_'
    postfix = '.txt'
    source = cwd + folder + file + postfix
        
    #export DataFrame to text file
    with open(source, 'a', encoding='utf-8') as f:
        for index, row in df.iterrows():        
            word_str = str(row['token'])
            value_str = str(row['frequency'])
            value_str = value_str + "\n"
            trimmed = word_str.lstrip()
            row_str = "\t".join([trimmed, value_str])
            f.write(row_str)
        
# Load dataframe to excel and csv at defined source file
load_N1grams_toCSV(df_results)
load_N1grams_toTXT(df_results)

# Lemma frequency analysis - as unigram analysis (1-gram analysis)

## Lemma frequency analysis for 'correct' HS samples

### Preprocessed by basic cleaning process
- leave only upper and lower characters
- tokenization
- lower
- leave only alphabet
- remove Finnish stopwords (Merja's list of stopwords)
- remove Finnish sentiment words (Merja's list of sentiment / emotion words)

In [29]:
# for dataset samples including hate speech - 'HS' category
df_HS

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]",--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]",-=>Widdu joo<= -,-= > vittu joo < = -
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,"[-, ei, viddu, -]","[ei, vittu]",- ei viddu -,ei vittu
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...","[-, Hyvä, että, JOKU, näkee, uuden, ,, tyhmän,...","[Hyvä, että, JOKU, näkee, uuden, ,, tyhmän, ve...","- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ..."
11,12,- Tarpeeksi vihainen mies tappaa nyrkillä kius...,2,-5,1,3,GROUP,FAMILY SCHOOL FRIENDS,"INCITEMENT, VIOLENCE","ANGER HATE, AGGRESSIVE VIOLENT",- Tarpeeksi vihainen mies tappaa nyrkillä kius...,Tarpeeksi vihainen mies tappaa nyrkillä kiusaa...,Meria,- Tarpeeksi vihainen mies tappaa nyrkillä kius...,Tarpeeksi vihainen mies tappaa nyrkillä kiusaa...,"[-, Tarpeeksi, vihainen, mies, tappaa, nyrkill...","[Tarpeeksi, vihainen, mies, tappaa, nyrkillä, ...",- tarpeeksi vihainen mies tappaa nyrkki kiusaa...,tarpeeksi vihainen mies tappaa nyrkki kiusaaja...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6611,6612,ööliä vittu,2,-4,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",CONTEMPT DISRESPECT,ööliä vittu,olutta vittu,Meria,ööliä vittu,olutta vittu,"[ööliä, vittu]","[olutta, vittu]",ööliä vittu,olut vittu
6612,6613,öö-luokkalaiset.vittuun,2,-5,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING",SARCASTIC,öö-luokkalaiset vittuun,öö-luokkalaiset vittuun,Meria,öö-luokkalaiset.vittuun,öö-luokkalaiset vittuun,"[öö-luokkalaiset, ., vittuun]","[öö-luokkalaiset, vittuun]",öö-luokkalaiset.vittuun,öö-luokkalaiset vittu
6613,6614,öö-vittu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",DISGUST,öö -vittu,öö vittu,Meria,öö-vittu,öö vittu,[öö-vittu],"[öö, vittu]",öö-vittu,öö vittu
6615,6616,Ööö mitä vittua??,2,-4,1,2,NONE,OPINION,"JOKE SARCASM, SWEARING",SURPRISE,Ööö mitä vittua??,"Öö, mitä vittua??",Meria,Ööö mitä vittua??,"Öö, mitä vittua??","[Ööö, mitä, vittua, ?, ?]","[Öö, ,, mitä, vittua, ?, ?]",Ööö mitä vittu ? ?,"Öö , mitä vittu ? ?"


In [30]:
# ------------------------------------------------------#
# collect all correct Finnish samples to one dict

allComments = []
for sample in df_HS['correctLemmas']:
    allComments.append(sample)
#print("All samples:  ", allComments)

#----------- lower characters -------------------------#
#----------- clean special chars and numbers ----------#

tokenized_comments = []
# cleaning - remove all non alphabets
regex = re.compile('[\W]+')
for comment in allComments:
    tokens = casual_tokenize(comment)
    tokens = [token.lower() for token in tokens]
    tokens = [regex.sub(' ', token) for token in tokens]
    tokens = [token for token in tokens if token.isalpha()]
    tokenized_comment = ' '.join(tokens)
    tokenized_comments.append(tokenized_comment)

# ------------ word tokenization -----------------------#
list_of_all_lemmas =" ".join(tokenized_comments)
lemma_tokens = word_tokenize(list_of_all_lemmas)
#print('\nList of tokenized samples:   ', lemma_tokens)

# ------------ remove stopwords ------------

#stopwordsFinnish = set(stopwords.words("finnish"))
stopwordsFinnish = FinnishStopwords
stopwordsRemoved = [word for word in lemma_tokens if word not in stopwordsFinnish]
#print('\nStopwords removed of all lemmas list\n')
#print(stopwordsRemoved)

# ------------ remove Finnish sentiment words ------------

sentimentsRemoved = []

sentimentsFinnish = finnish_sentiment_words
sentimentsRemoved = [word for word in stopwordsRemoved if word not in sentimentsFinnish]

sentimentsRemoved = []

for word in stopwordsRemoved:
    if word not in sentimentsFinnish:
        sentimentsRemoved.append(word)

print('\nLen of sentiment words removed:   ', len(sentimentsRemoved))
print('\nSentiment words removed of all words list\n')
print(sentimentsRemoved)



Len of sentiment words removed:    3743

Sentiment words removed of all words list

['veroperuste', 'co', 'diesel', 'noki', 'ilma', 'keuhko', 'mitta', 'täyteen', 'vittutieteen', 'tohtori', 'käristyä', 'vastaavasti', 'pukea', 'palli', 'johan', 'kauhistella', 'naurismaa', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'mopar', 'willimiehen', 'disc', 'golf', 'viljellä', 'kokoaika', 'pienimuotoinen', 'olo', 'abortööri', 'seksuaalisesti', 'porno', 'viljellä', 'vittuuntumatta', 'sekunti', 'mahdollisesti', 'ylikiltti', 'puola', 'koulupoika', 'ikäluokka', 'prosenttinen', 'vauva', 'alkaen', 'tökerö', 'argumentaatiovirhe', 'uimahalli', 'vesi', 'tunti', 'vuokraisäntä', 'vaahdota', 'vuokra', 'asuminen', 'vuokra', 'vuokratuotto', 'intressi', 'vuokralainen', 'vuotias', 'kittivittu', 'mikähän', 'tullu', 'niko', 'fir', 'fllt', 'frilttaillaan', 'puhevuoro', 'kuulo', 'erityinen', 'patologi

In [31]:
# Use counter to count lemmas unigrams
N1gram = Counter(sentimentsRemoved)

# create data frame for lemma unigrams
df_results = pd.DataFrame(N1gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)
df_results.head(50)

,lemma,frequency
221,matu,71
124,ulos,27
50,vuotias,22
17,mopar,19
792,sipilä,17
118,dd,14
145,vuo,14
140,vitutus,13
554,matuille,12
116,d,12


In [32]:
# load dataframe to csv at at output folder

def load_N1grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_HS_lemmas_N1gram_CSV_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to txt at output folder
def load_N1grams_toTXT(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    file = 'fin_HS_lemmas_N1gram_TXT_'
    postfix = '.txt'
    source = cwd + folder + file + postfix
        
    #export DataFrame to text file
    with open(source, 'a', encoding='utf-8') as f:
        for index, row in df.iterrows():        
            word_str = str(row['lemma'])
            value_str = str(row['frequency'])
            value_str = value_str + "\n"
            trimmed = word_str.lstrip()
            row_str = "\t".join([trimmed, value_str])
            f.write(row_str)
        
# Load dataframe to txt and csv at defined source file
load_N1grams_toCSV(df_results)
load_N1grams_toTXT(df_results)


## Lemma frequency analysis for 'correct' 'notHS' samples

### Preprocessed by basic cleaning process
- leave only upper and lower characters
- tokenization
- lower
- leave only alphabet
- remove Finnish stopwords (Merja's list of stopwords)
- remove Finnish sentiment words (Merja's list of sentiment / emotion words)

In [33]:
# for dataset samples not including hate speech - 'notHS' category
df_notHS = df[df['HSbinary'] <= 0]
df_notHS

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,"[-, Ajatus, siitä, että, kaikki, henkilön, tie...","[-, Ajatus, siitä, että, kaikki, henkilön, tie...",- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ..."
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...","[-, Ei, ,, mutta, olen, joutunut, elämään, kat...","[Ei, ,, mutta, olen, joutunut, elämään, katsom...","- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius..."
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,"[-, Huvittunut, -]",[huvittunut],- huvittunut -,huvittua
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,"[-, Kukaan, luterilainen, ei, korota, itseään,...","[Kukaan, luterilainen, ei, korota, itseään, hä...",- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,1,5,0,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,"[😂, 😂, 😂]","[😂, 😂, 😂]",😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂,😂 😂 😂 😂 😂
6625,6626,😠👉👩‍💼,2,-3,0,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria,😠👉👩‍💼,😠👉👩‍💼,"[😠, 👉, 👩‍💼]","[😠, 👉, 👩‍💼]",😠 👉 👩 ‍ 💼,😠 👉 👩 ‍ 💼
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,1,5,0,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,"[🤣, 🤣, 🤣]","[🤣, 🤣, 🤣]",🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣,🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣
6627,6628,🤮😡,2,-4,0,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria,🤮😡,🤮😡,"[🤮, 😡]","[🤮, 😡]",🤮 😡,🤮 😡


In [34]:
# ------------------------------------------------------#
# collect all correct Finnish samples to one dict

allComments = []
for sample in df_notHS['correctLemmas']:
    allComments.append(sample)
#print("All samples:  ", allComments)

#----------- lower characters -------------------------#
#----------- clean special chars and numbers ----------#

tokenized_comments = []
# cleaning - remove all non alphabets
regex = re.compile('[\W]+')
for comment in allComments:
    tokens = casual_tokenize(comment)
    tokens = [token.lower() for token in tokens]
    tokens = [regex.sub(' ', token) for token in tokens]
    tokens = [token for token in tokens if token.isalpha()]
    tokenized_comment = ' '.join(tokens)
    tokenized_comments.append(tokenized_comment)

# ------------ word tokenization -----------------------#
list_of_all_lemmas =" ".join(tokenized_comments)
lemma_tokens = word_tokenize(list_of_all_lemmas)
#print('\nList of tokenized samples:   ', lemma_tokens)

# ------------ remove stopwords ------------

#stopwordsFinnish = set(stopwords.words("finnish"))
stopwordsFinnish = FinnishStopwords
stopwordsRemoved = [word for word in lemma_tokens if word not in stopwordsFinnish]
#print('\nStopwords removed of all lemmas list\n')
#print(stopwordsRemoved)

# ------------ remove Finnish sentiment words ------------

sentimentsRemoved = []

sentimentsFinnish = finnish_sentiment_words
sentimentsRemoved = [word for word in stopwordsRemoved if word not in sentimentsFinnish]

sentimentsRemoved = []

for word in stopwordsRemoved:
    if word not in sentimentsFinnish:
        sentimentsRemoved.append(word)

print('\nLen of sentiment words removed:   ', len(sentimentsRemoved))
print('\nSentiment words removed of all words list\n')
print(sentimentsRemoved)



Len of sentiment words removed:    5491

Sentiment words removed of all words list

['lähinnä', 'hyvinvointiyhteiskunta', 'lepomäki', 'terveydenhoito', 'koulutus', 'huristaa', 'haastattelu', 'verotus', 'vastaava', 'tienattu', 'lantti', 'yksilö', 'sijoittaa', 'kotimaa', 'konkreettisesti', 'kotimaa', 'myynti', 'työntekijä', 'tossa', 'vasemmisto', 'kansantalous', 'toiminta', 'kannin', 'kaivovertaus', 'viime', 'kaivaa', 'ylhäällä', 'olija', 'keittiöjakkara', 'seistä', 'lepomäki', 'tuloverka', 'olettaa', 'kyse', 'valtionvero', 'tuloverotus', 'toki', 'kunnallisvero', 'tasavero', 'keskiluokka', 'valtaosa', 'kokonaisuus', 'verotus', 'progressio', 'lievä', 'toki', 'kokonaisverotuksessa', 'verka', 'valtionvelka', 'valtionvelka', 'velanotto', 'voitollinen', 'toiminta', 'kansainvälinen', 'vertailu', 'vähävelkainen', 'läheinen', 'mureta', 'auttaakse', 'korottaa', 'runnoa', 'pohjamuta', 'ylipäänsä', 'jompi', 'kumpi', 'kumpikin', 'välimatka', 'sisarus', 'exän', 'kynnys', 'vieläkään', 'pelleillä', 'p

In [35]:
# Use counter to count lemmas unigrams
N1gram = Counter(sentimentsRemoved)

# create data frame for lemma unigrams
df_results = pd.DataFrame(N1gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)
df_results.head(30)

,lemma,frequency
171,video,44
339,vuotias,40
70,vuo,33
642,tosi,29
31,kyse,22
221,eräs,19
2,lepomäki,18
973,fazer,18
185,xd,16
141,kuulostaa,15


In [36]:
# load dataframe to csv at at output folder

def load_N1grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_notHS_lemmas_N1gram_CSV_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to txt at output folder
def load_N1grams_toTXT(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    file = 'fin_notHS_lemmas_N1gram_TXT_'
    postfix = '.txt'
    source = cwd + folder + file + postfix
        
    #export DataFrame to text file
    with open(source, 'a', encoding='utf-8') as f:
        for index, row in df.iterrows():        
            word_str = str(row['lemma'])
            value_str = str(row['frequency'])
            value_str = value_str + "\n"
            trimmed = word_str.lstrip()
            row_str = "\t".join([trimmed, value_str])
            f.write(row_str)
        
# Load dataframe to txt and csv at defined source file
load_N1grams_toCSV(df_results)
load_N1grams_toTXT(df_results)

# Lemma frequency analysis for 'positive' and 'negative' sentiment classes

## Lemma frequency analysis for 'positive' sentiment samples

### Preprocessed by basic cleaning process
- leave only upper and lower characters
- tokenization
- lower
- leave only alphabet
- remove Finnish stopwords (Merja's list of stopwords)
- remove Finnish sentiment words (Merja's list of sentiment / emotion words)

In [37]:
# stopwords filtered
# sentiment words filtered
# significant words on sentiment words list?
df

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,"[-, Ajatus, siitä, että, kaikki, henkilön, tie...","[-, Ajatus, siitä, että, kaikki, henkilön, tie...",- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]",--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]",-=>Widdu joo<= -,-= > vittu joo < = -
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...","[-, Ei, ,, mutta, olen, joutunut, elämään, kat...","[Ei, ,, mutta, olen, joutunut, elämään, katsom...","- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,1,5,0,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,"[😂, 😂, 😂]","[😂, 😂, 😂]",😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂,😂 😂 😂 😂 😂
6625,6626,😠👉👩‍💼,2,-3,0,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria,😠👉👩‍💼,😠👉👩‍💼,"[😠, 👉, 👩‍💼]","[😠, 👉, 👩‍💼]",😠 👉 👩 ‍ 💼,😠 👉 👩 ‍ 💼
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,1,5,0,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,"[🤣, 🤣, 🤣]","[🤣, 🤣, 🤣]",🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣,🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣
6627,6628,🤮😡,2,-4,0,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria,🤮😡,🤮😡,"[🤮, 😡]","[🤮, 😡]",🤮 😡,🤮 😡


In [38]:
df_positive = df[df['sentiment'] == 1]
df_positive

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,"[-, Huvittunut, -]",[huvittunut],- huvittunut -,huvittua
25,26,!huvittunut!,1,2,0,0,NaN,NaN,NaN,JOY,! huvittunut!,! huvittunut!,Meria,!huvittunut!,! huvittunut!,"[!, huvittunut, !]","[!, huvittunut, !]",! huvittua !,! huvittua !
46,47,""" Mikset sinäkin ottaisi vastaan Timo Jutilaa ...",1,3,0,0,NaN,NaN,NaN,JOY,""" Mikset sinäkin ottaisi vastaan Timo Jutilaa ...","""Mikset sinäkin ottaisi vastaan Timo Jutilaa j...",Meria,""" Mikset sinäkin ottaisi vastaan Timo Jutilaa ...","""Mikset sinäkin ottaisi vastaan Timo Jutilaa j...","["", Mikset, sinäkin, ottaisi, vastaan, Timo, J...","["", Mikset, sinäkin, ottaisi, vastaan, Timo, J...",""" miks ei sinä ottaa vastaan Timo Jutila ja Ta...",""" miks ei sinä ottaa vastaan Timo Jutila ja Ta..."
87,88,"""Hei vaan teille ja onnittelut"" 😂😂😭",1,2,0,0,NaN,NaN,NaN,"MIXED EMOTIONS, JOY, SADNESS, SARCASTIC","""Hei vaan teille ja onnittelut"" 😂😂😭","""Hei vaan teille ja onnittelut"" 😂😂😭",Meria,"""Hei vaan teille ja onnittelut"" 😂😂😭","""Hei vaan teille ja onnittelut"" 😂😂😭","["", Hei, vaan, teille, ja, onnittelut, "", 😂, 😂...","["", Hei, vaan, teille, ja, onnittelut, "", 😂, 😂...",""" hei vaan sinä ja onnittelu "" 😂 😂 😭",""" hei vaan sinä ja onnittelu "" 😂 😂 😭"
90,91,"""ikärajoitettu video"" 😂😂",1,4,0,0,NaN,NaN,NaN,"JOY, SARCASTIC","""ikärajoitettu video"" 😂😂","""ikärajoitettu video"" 😂😂",Meria,"""ikärajoitettu video"" 😂😂","""ikärajoitettu video"" 😂😂","["", ikärajoitettu, video, "", 😂, 😂]","["", ikärajoitettu, video, "", 😂, 😂]",""" ikärajoittaa video "" 😂 😂",""" ikärajoittaa video "" 😂 😂"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6621,6622,😂,1,2,0,0,NaN,NaN,NaN,JOY,😂,😂,Meria,😂,😂,[😂],[😂],😂,😂
6623,6624,😂😂😂😂😂😂,1,5,0,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂,😂😂😂😂😂😂,Meria,😂😂😂😂😂😂,😂😂😂😂😂😂,"[😂, 😂, 😂]","[😂, 😂, 😂]",😂 😂 😂 😂 😂 😂,😂 😂 😂 😂 😂 😂
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,1,5,0,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,"[😂, 😂, 😂]","[😂, 😂, 😂]",😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂,😂 😂 😂 😂 😂
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,1,5,0,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,"[🤣, 🤣, 🤣]","[🤣, 🤣, 🤣]",🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣,🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣


In [39]:
# ------------------------------------------------------#
# collect all correct Finnish samples to one dict

allComments = []
for sample in df_positive['correctLemmas']:
    allComments.append(sample)
#print("All samples:  ", allComments)

#----------- lower characters -------------------------#
#----------- clean special chars and numbers ----------#

tokenized_comments = []
# cleaning - remove all non alphabets
regex = re.compile('[\W]+')
for comment in allComments:
    tokens = casual_tokenize(comment)
    tokens = [token.lower() for token in tokens]
    tokens = [regex.sub(' ', token) for token in tokens]
    tokens = [token for token in tokens if token.isalpha()]
    tokenized_comment = ' '.join(tokens)
    tokenized_comments.append(tokenized_comment)

# ------------ word tokenization -----------------------#
list_of_all_lemmas =" ".join(tokenized_comments)
lemma_tokens = word_tokenize(list_of_all_lemmas)
#print('\nList of tokenized samples:   ', lemma_tokens)

# ------------ remove stopwords ------------

#stopwordsFinnish = set(stopwords.words("finnish"))
stopwordsFinnish = FinnishStopwords
stopwordsRemoved = [word for word in lemma_tokens if word not in stopwordsFinnish]
#print('\nStopwords removed of all lemmas list\n')
#print(stopwordsRemoved)

# ------------ remove Finnish sentiment words ------------

sentimentsRemoved = []

sentimentsFinnish = finnish_sentiment_words
sentimentsRemoved = [word for word in stopwordsRemoved if word not in sentimentsFinnish]

sentimentsRemoved = []

for word in stopwordsRemoved:
    if word not in sentimentsFinnish:
        sentimentsRemoved.append(word)

print('\nLen of sentiment words removed:   ', len(sentimentsRemoved))
print('\nSentiment words removed of all words list\n')
print(sentimentsRemoved)


Len of sentiment words removed:    870

Sentiment words removed of all words list

['timo', 'jutila', 'tappara', 'onnittelu', 'video', 'xd', 'xd', 'ps', 'video', 'normaalipainoinen', 'tappara', 'kalpa', 'kalpa', 'joukkue', 'eräs', 'lapsuus', 'stouth', 'bitter', 'ydinvoima', 'arto', 'lauri', 'youtube', 'kanava', 'tilattava', 'pöytä', 'valmistella', 'suklaakakkutarjoilu', 'kirjoittaja', 'hiihto', 'vuokatti', 'hiihdellä', 'lumi', 'latu', 'paanat', 'ilma', 'heikkilä', 'j', 'metsikkö', 'luonto', 'dd', 'dddd', 'dddd', 'musiikki', 'kuris', 'kirjoitus', 'pääomatulo', 'xd', 'tilaaja', 'xd', 'frlirlr', 'sokkotreffi', 'ulkomaa', 'viikonloppu', 'unkari', 'ulkonäkö', 'xddd', 'km', 'rajotin', 'vieläkin', 'x', 'xd', 'vuotias', 'vuotias', 'vuo', 'olo', 'olo', 'suolalamppu', 't', 'elina', 'chisu', 'keikka', 'muodostaa', 'vlc', 'aikaisemmin', 'luokse', 'asunto', 'lukija', 'erkki', 'kiinostunutkaan', 'tanssikuvio', 'jonsu', 'teatterikorkeeseekin', 'tosisaan', 'tutkija', 'haaponivan', 'kilometri', 'kesku

In [40]:
#count the lemma, lemma 2-grams, and lemma trigrams
N1gram = Counter(sentimentsRemoved)

# create data frame for lemmas
df_results = pd.DataFrame(N1gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)
df_results.head(50)

,lemma,frequency
4,video,21
5,xd,14
94,tosi,12
95,hjallis,10
54,vuo,9
405,jussi,6
121,ddd,5
614,ääretön,5
189,aamu,5
148,kaksi,5


In [41]:
# load dataframe to csv at at output folder

def load_N1grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_positive_lemmas_N1gram_CSV_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to txt at output folder
def load_N1grams_toTXT(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    file = 'fin_positive_lemmas_N1gram_TXT_'
    postfix = '.txt'
    source = cwd + folder + file + postfix
        
    #export DataFrame to text file
    with open(source, 'a', encoding='utf-8') as f:
        for index, row in df.iterrows():        
            word_str = str(row['lemma'])
            value_str = str(row['frequency'])
            value_str = value_str + "\n"
            trimmed = word_str.lstrip()
            row_str = "\t".join([trimmed, value_str])
            f.write(row_str)
        
# Load dataframe to txt and csv at defined source file
load_N1grams_toCSV(df_results)
load_N1grams_toTXT(df_results)

## Lemma frequency analysis for 'negative' sentiment samples

### Preprocessed by basic cleaning process
- leave only upper and lower characters
- tokenization
- lower
- leave only alphabet
- remove Finnish stopwords (Merja's list of stopwords)
- remove Finnish sentiment words (Merja's list of sentiment / emotion words)

In [42]:
df_negative = df[df['sentiment'] == 2]
df_negative

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,"[-, Ajatus, siitä, että, kaikki, henkilön, tie...","[-, Ajatus, siitä, että, kaikki, henkilön, tie...",- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]",--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]",-=>Widdu joo<= -,-= > vittu joo < = -
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...","[-, Ei, ,, mutta, olen, joutunut, elämään, kat...","[Ei, ,, mutta, olen, joutunut, elämään, katsom...","- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6616,6617,Ööövittuentiiä,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",CONTEMPT DISRESPECT,Ööö vittu en tiiä,Öö vittu en tiedä,Meria,Ööövittuentiiä,Öö vittu en tiedä,[Ööövittuentiiä],"[Öö, vittu, en, tiedä]",ööövittuentiiä,Öö vittu ei tietää
6619,6620,👎😠,2,-3,0,0,NaN,NaN,NaN,"UNPLEASENT, ANGER HATE",👎😠,👎😠,Meria,👎😠,👎😠,"[👎, 😠]","[👎, 😠]",👎 😠,👎 😠
6622,6623,😂 ai kauhee. Ei oo hauska asia jos itelle tapa...,2,-4,0,0,NaN,NaN,NaN,SURPRISE,😂 ai kauhee. Ei oo hauska asia jos itelle tapa...,😂 ai kauhee. Ei oo hauska asia jos itelle tapa...,Meria,😂 ai kauhee. Ei oo hauska asia jos itelle tapa...,😂 ai kauhee. Ei oo hauska asia jos itelle tapa...,"[😂, ai, kauhee, ., Ei, oo, hauska, asia, jos, ...","[😂, ai, kauhee, ., Ei, oo, hauska, asia, jos, ...",😂 ai kauhee . ei oo hauska asia jos itelle tap...,😂 ai kauhee . ei oo hauska asia jos itelle tap...
6625,6626,😠👉👩‍💼,2,-3,0,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria,😠👉👩‍💼,😠👉👩‍💼,"[😠, 👉, 👩‍💼]","[😠, 👉, 👩‍💼]",😠 👉 👩 ‍ 💼,😠 👉 👩 ‍ 💼


In [43]:
# ------------------------------------------------------#
# collect all correct Finnish samples to one dict

allComments = []
for sample in df_negative['correctLemmas']:
    allComments.append(sample)
#print("All samples:  ", allComments)

#----------- lower characters -------------------------#
#----------- clean special chars and numbers ----------#

tokenized_comments = []
# cleaning - remove all non alphabets
regex = re.compile('[\W]+')
for comment in allComments:
    tokens = casual_tokenize(comment)
    tokens = [token.lower() for token in tokens]
    tokens = [regex.sub(' ', token) for token in tokens]
    tokens = [token for token in tokens if token.isalpha()]
    tokenized_comment = ' '.join(tokens)
    tokenized_comments.append(tokenized_comment)

# ------------ word tokenization -----------------------#
list_of_all_lemmas =" ".join(tokenized_comments)
lemma_tokens = word_tokenize(list_of_all_lemmas)
#print('\nList of tokenized samples:   ', lemma_tokens)

# ------------ remove stopwords ------------

#stopwordsFinnish = set(stopwords.words("finnish"))
stopwordsFinnish = FinnishStopwords
stopwordsRemoved = [word for word in lemma_tokens if word not in stopwordsFinnish]
print('\nStopwords removed of all lemmas list\n')
print(stopwordsRemoved)

# ------------ remove Finnish sentiment words ------------

sentimentsRemoved = []

sentimentsFinnish = finnish_sentiment_words
sentimentsRemoved = [word for word in stopwordsRemoved if word not in sentimentsFinnish]

sentimentsRemoved = []

for word in stopwordsRemoved:
    if word not in sentimentsFinnish:
        sentimentsRemoved.append(word)

print('\nLen of sentiment words removed:   ', len(sentimentsRemoved))
print('\nSentiment words removed of all words list\n')
print(sentimentsRemoved)


Stopwords removed of all lemmas list

['ajatus', 'kaikki', 'henkilö', 'tienata', 'tulo', 'oma', 'työ', 'tulos', 'lähinnä', 'naurettava', 'voida', 'ehkä', 'päteä', 'henkilö', 'elää', 'yksin', 'metsä', 'susi', 'kasvattaa', 'ei', 'moderni', 'hyvinvointiyhteiskunta', 'yhteiskunta', 'maksaa', 'lepomäki', 'terveydenhoito', 'koulutus', 'rakentaa', 'tie', 'joki', 'huristaa', 'haastattelu', 'yritys', 'ei', 'voida', 'toimia', 'ilman', 'verotus', 'vastaava', 'syy', 'jokainen', 'tienattu', 'lantti', 'mukana', 'todella', 'suuri', 'osa', 'yhteiskunta', 'yksilö', 'yritys', 'sijoittaa', 'raha', 'kotimaa', 'yritys', 'tapaus', 'ihan', 'konkreettisesti', 'kotimaa', 'myynti', 'maksaa', 'veroraha', 'julkinen', 'puoli', 'työntekijä', 'palkka', 'tossa', 'muu', 'ei', 'jaksaa', 'keskustelu', 'peruste', 'vasemmisto', 'ymmärrys', 'yhteiskunta', 'kansantalous', 'toiminta', 'hyvä', 'kannin', 'oikeisto', 'kaivovertaus', 'viime', 'hallitus', 'kaivaa', 'kaivo', 'köyhä', 'seisoa', 'vielä', 'hieman', 'syvä', 'antaa', 

In [44]:
#count the lemma, lemma 2-grams, and lemma trigrams
N1gram = Counter(sentimentsRemoved)

# create data frame for lemmas
lemma_freq_N1 = pd.DataFrame(N1gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)
lemma_freq_N1.head(50)

,lemma,frequency
465,matu,71
333,ulos,31
91,vuo,29
1461,sipilä,28
2,lepomäki,27
179,vuotias,23
87,mopar,19
880,kai,18
345,tosi,18
31,kyse,18


In [45]:
# load dataframe to csv at at output folder

def load_N1grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_negative_lemmas_N1gram_CSV_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to txt at output folder
def load_N1grams_toTXT(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    file = 'fin_negative_lemmas_N1gram_TXT_'
    postfix = '.txt'
    source = cwd + folder + file + postfix
        
    #export DataFrame to text file
    with open(source, 'a', encoding='utf-8') as f:
        for index, row in df.iterrows():        
            word_str = str(row['lemma'])
            value_str = str(row['frequency'])
            value_str = value_str + "\n"
            trimmed = word_str.lstrip()
            row_str = "\t".join([trimmed, value_str])
            f.write(row_str)
        
# Load dataframe to txt and csv at defined source file
load_N1grams_toCSV(lemma_freq_N1)
load_N1grams_toTXT(lemma_freq_N1)

# Lemma frequency analysis for 
# 'HS strength' subclasses '-4' & '-5', and 
# 'HS strength' subclasses '-1' & '-2' 

## Lemma frequency analysis for HSstrength '-4' & '-5' subcategory samples

### Preprocessed by basic cleaning process
- leave only upper and lower characters
- tokenization
- lower
- leave only alphabet
- remove Finnish stopwords (Merja's list of stopwords)

In [46]:
df

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,"[-, Ajatus, siitä, että, kaikki, henkilön, tie...","[-, Ajatus, siitä, että, kaikki, henkilön, tie...",- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]",--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]",-=>Widdu joo<= -,-= > vittu joo < = -
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...","[-, Ei, ,, mutta, olen, joutunut, elämään, kat...","[Ei, ,, mutta, olen, joutunut, elämään, katsom...","- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,1,5,0,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,"[😂, 😂, 😂]","[😂, 😂, 😂]",😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂,😂 😂 😂 😂 😂
6625,6626,😠👉👩‍💼,2,-3,0,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria,😠👉👩‍💼,😠👉👩‍💼,"[😠, 👉, 👩‍💼]","[😠, 👉, 👩‍💼]",😠 👉 👩 ‍ 💼,😠 👉 👩 ‍ 💼
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,1,5,0,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,"[🤣, 🤣, 🤣]","[🤣, 🤣, 🤣]",🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣,🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣
6627,6628,🤮😡,2,-4,0,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria,🤮😡,🤮😡,"[🤮, 😡]","[🤮, 😡]",🤮 😡,🤮 😡


In [47]:
df_HS45 = df[df['HSstrength'] >= 4]
df_HS45

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
18,19,!! ! mopar on paska paikka jossa on noobit hom...,2,-5,1,5,GROUP,"SEXUAL, GENDER, FAMILY SCHOOL FRIENDS, TROL...","INSULT, SWEARING, BULLY, GRANULATED","CONTEMPT DISRESPECT, ANGER HATE",!! ! mopar on paska paikka jossa on noobit hom...,!! ! mopar on paska paikka jossa on noobit hom...,Meria,!! ! mopar on paska paikka jossa on noobit hom...,!! ! mopar on paska paikka jossa on noobit hom...,"[!, !, !, mopar, on, paska, paikka, jossa, on,...","[!, !, !, mopar, on, paska, paikka, jossa, on,...",! ! ! mopar olla paska paikka joka olla noobit...,! ! ! mopar olla paska paikka joka olla noobit...
35,36,""" Ime isäsi KYRPÄÄ .",2,-5,1,5,PERSON,"SEXUAL, FAMILY SCHOOL FRIENDS","INSULT, HARASSMENT, SWEARING","DISGUST, CONTEMPT DISRESPECT",Ime isäsi KYRPÄÄ.,Ime isäsi kyrpää.,Meria,""" Ime isäsi KYRPÄÄ .",Ime isäsi kyrpää.,"["", Ime, isäsi, KYRPÄÄ, .]","[Ime, isäsi, kyrpää, .]",""" Imeä isä Kyrpä .",Imeä isä kyrpä .
107,108,"""Tiedän sijaintisi, sinä korruptoitunut huora....",2,-5,1,5,PERSON,"WORK, SEXUAL, GENDER, WOMEN, STATUS","THREAT, HARASSMENT, VIOLENCE, BULLY","ANGER HATE, AGGRESSIVE VIOLENT","""Tiedän sijaintisi, sinä korruptoitunut huora....","""Tiedän sijaintisi, sinä korruptoitunut huora....",Meria,"""Tiedän sijaintisi, sinä korruptoitunut huora....","""Tiedän sijaintisi, sinä korruptoitunut huora....","["", Tiedän, sijaintisi, ,, sinä, korruptoitunu...","["", Tiedän, sijaintisi, ,, sinä, korruptoitunu...",""" tietää sijainti , se korruptoida huora . tap...",""" tietää sijainti , se korruptoida huora . tap..."
431,432,>PILLU HALLITUS >PILLUHALLITUS >PILLU HALLITUS...,2,-5,1,5,PERSON,"POLITICS, SEXUAL, GENDER, WOMEN, TROLLING","INSULT, HARASSMENT, TARGETING, SWEARING, B...","DISGUST, CONTEMPT DISRESPECT",>PILLU HALLITUS >PILLUHALLITUS >PILLU HALLITUS...,PILLU HALLITUS PILLUHALLITUS PILLU HALLITUS OH...,Meria,>PILLU HALLITUS >PILLUHALLITUS >PILLU HALLITUS...,PILLU HALLITUS PILLUHALLITUS PILLU HALLITUS OH...,"[>, PILLU, HALLITUS, >, PILLUHALLITUS, >, PILL...","[PILLU, HALLITUS, PILLUHALLITUS, PILLU, HALLIT...",> pillu hallitus > pilluhallitus > pillu halli...,pillu hallitus pilluhallitus Pillu hallitus Oh...
432,433,">öyh hiiva pillut,, hullu akka marin,,,",2,-5,1,4,PERSON,"POLITICS, SEXUAL, GENDER, WOMEN, TROLLING","INSULT, HARASSMENT, TARGETING, SWEARING, B...","DISGUST, CONTEMPT DISRESPECT","öyh hiivapillut, hullu akka marin","öyh hiivapillut, hullu akka Marin",Meria,">öyh hiiva pillut,, hullu akka marin,,,","öyh hiivapillut, hullu akka Marin","[>, öyh, hiiva, pillut, ,, ,, hullu, akka, mar...","[öyh, hiivapillut, ,, hullu, akka, Marin]","> öyh hiiva pillu , , hullu akka Marin , , ,","öyh hiivapillu , hullu akka Marin"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779,5780,VitunVitunVitunVittua,2,-5,1,4,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,Vitun Vitun Vitun Vittua,vitun vitun vitun vittua,Meria,VitunVitunVitunVittua,vitun vitun vitun vittua,[VitunVitunVitunVittua],"[vitun, vitun, vitun, vittua]",vitunvitunvitunvittua,vittu vittu vittu vittu
5780,5781,Vitunvitunvitunvitunvttu,2,-5,1,5,NONE,OTHER,"SWEARING, GRANULATED","CONTEMPT DISRESPECT, ANGER HATE",Vitun vitun vitun vitun vttu,vitun vitun vitun vitun vittu,Meria,Vitunvitunvitunvitunvttu,vitun vitun vitun vitun vittu,[Vitunvitunvitunvitunvttu],"[vitun, vitun, vitun, vitun, vittu]",vitunvitunvitunvitunvttu,vittu vittu vittu vittu vittu
5786,5787,vituvittuvittuvittuvittu,2,-5,1,5,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,vitu vittu vittu vittu vittu,vittu vittu vittu vittu vittu,Meria,vituvittuvittuvittuvittu,vittu vittu vittu vittu vittu,[vituvittuvittuvittuvittu],"[vittu, vittu, vittu, vittu, vittu]",vituvittuvittuvittuvittu,vittu vittu vittu vittu vittu
6255,6256,ylitäkuolemanraja,2,-3,1,4,PERSON,TROLLING,"HARAS

In [48]:
# ------------------------------------------------------#
# collect all correct Finnish samples to one dict

allComments = []
for sample in df_HS45['sampleLemmas']:
    allComments.append(sample)
#print("All samples:  ", allComments)

#----------- lower characters -------------------------#
#----------- clean special chars and numbers ----------#

tokenized_comments = []
# cleaning - remove all non alphabets
regex = re.compile('[\W]+')
for comment in allComments:
    tokens = casual_tokenize(comment)
    tokens = [token.lower() for token in tokens]
    tokens = [regex.sub(' ', token) for token in tokens]
    tokens = [token for token in tokens if token.isalpha()]
    tokenized_comment = ' '.join(tokens)
    tokenized_comments.append(tokenized_comment)

# ------------ word tokenization -----------------------#
list_of_all_lemmas =" ".join(tokenized_comments)
lemma_tokens = word_tokenize(list_of_all_lemmas)
#print('\nList of tokenized samples:   ', lemma_tokens)

# ------------ remove stopwords ------------

#stopwordsFinnish = set(stopwords.words("finnish"))
stopwordsFinnish = FinnishStopwords
stopwordsRemoved = [word for word in lemma_tokens if word not in stopwordsFinnish]
#print('\nStopwords removed of all lemmas list\n')
#print(stopwordsRemoved)

#count the lemma, lemma 2-grams, and lemma trigrams
N1gram = Counter(stopwordsRemoved)

# create data frame for lemmas
lemma_freq_N1 = pd.DataFrame(N1gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)
lemma_freq_N1.head(50)


,lemma,frequency
1,paska,110
228,ime,83
41,vittu,66
51,ei,33
4,homo,26
0,mopar,19
2,paikka,19
3,noobit,19
7,kyrpä,11
241,saatana,11


In [49]:
# load dataframe to csv at at output folder

def load_N1grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_HSstrength45_lemmas_N1gram_CSV_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to txt at output folder
def load_N1grams_toTXT(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    file = 'fin_HSstrength45_lemmas_N1gram_TXT_'
    postfix = '.txt'
    source = cwd + folder + file + postfix
        
    #export DataFrame to text file
    with open(source, 'a', encoding='utf-8') as f:
        for index, row in df.iterrows():        
            word_str = str(row['lemma'])
            value_str = str(row['frequency'])
            value_str = value_str + "\n"
            trimmed = word_str.lstrip()
            row_str = "\t".join([trimmed, value_str])
            f.write(row_str)
        
# Load dataframe to txt and csv at defined source file
load_N1grams_toCSV(lemma_freq_N1)
load_N1grams_toTXT(lemma_freq_N1)

## Lemma frequency analysis for HSstrength '-4' & '-5' subcategory samples

### Preprocessed by basic cleaning process
- leave only upper and lower characters
- tokenization
- lower
- leave only alphabet
- remove Finnish stopwords (Merja's list of stopwords)

In [50]:
df_HS12 = df[df['HSstrength'] <= 2]
df_HS12

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,sampleTokens,correctTokens,sampleLemmas,correctLemmas
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,"[-, Ajatus, siitä, että, kaikki, henkilön, tie...","[-, Ajatus, siitä, että, kaikki, henkilön, tie...",- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","[-, Kaivovertauksessa, ,, viime, hallitus, on,...","- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]",--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]",-=>Widdu joo<= -,-= > vittu joo < = -
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...","[-, Ei, ,, mutta, olen, joutunut, elämään, kat...","[Ei, ,, mutta, olen, joutunut, elämään, katsom...","- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,1,5,0,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,"[😂, 😂, 😂]","[😂, 😂, 😂]",😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂,😂 😂 😂 😂 😂
6625,6626,😠👉👩‍💼,2,-3,0,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria,😠👉👩‍💼,😠👉👩‍💼,"[😠, 👉, 👩‍💼]","[😠, 👉, 👩‍💼]",😠 👉 👩 ‍ 💼,😠 👉 👩 ‍ 💼
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,1,5,0,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,"[🤣, 🤣, 🤣]","[🤣, 🤣, 🤣]",🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣,🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣 🤣
6627,6628,🤮😡,2,-4,0,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria,🤮😡,🤮😡,"[🤮, 😡]","[🤮, 😡]",🤮 😡,🤮 😡


In [51]:
# ------------------------------------------------------#
# collect all correct Finnish samples to one dict

allComments = []
for sample in df_HS12['sampleLemmas']:
    allComments.append(sample)
#print("All samples:  ", allComments)

#----------- lower characters -------------------------#
#----------- clean special chars and numbers ----------#

tokenized_comments = []
# cleaning - remove all non alphabets
regex = re.compile('[\W]+')
for comment in allComments:
    tokens = casual_tokenize(comment)
    tokens = [token.lower() for token in tokens]
    tokens = [regex.sub(' ', token) for token in tokens]
    tokens = [token for token in tokens if token.isalpha()]
    tokenized_comment = ' '.join(tokens)
    tokenized_comments.append(tokenized_comment)

# ------------ word tokenization -----------------------#
list_of_all_lemmas =" ".join(tokenized_comments)
lemma_tokens = word_tokenize(list_of_all_lemmas)
#print('\nList of tokenized samples:   ', lemma_tokens)

# ------------ remove stopwords ------------

#stopwordsFinnish = set(stopwords.words("finnish"))
stopwordsFinnish = FinnishStopwords
stopwordsRemoved = [word for word in lemma_tokens if word not in stopwordsFinnish]
#print('\nStopwords removed of all lemmas list\n')
#print(stopwordsRemoved)

#count the lemma, lemma 2-grams, and lemma trigrams
N1gram = Counter(stopwordsRemoved)

# create data frame for lemmas
lemma_freq_N1 = pd.DataFrame(N1gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)
lemma_freq_N1.head(50)


,lemma,frequency
18,ei,1572
239,vittu,871
269,ihana,266
10,voida,240
1,kaikki,238
66,hyvä,228
112,saada,213
206,ihminen,199
253,tulla,184
144,tehdä,177


In [52]:
# load dataframe to csv at at output folder

def load_N1grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_HSstrength12_lemmas_N1gram_CSV_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to txt at output folder
def load_N1grams_toTXT(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    file = 'fin_HSstrength12_lemmas_N1gram_TXT_'
    postfix = '.txt'
    source = cwd + folder + file + postfix
        
    #export DataFrame to text file
    with open(source, 'a', encoding='utf-8') as f:
        for index, row in df.iterrows():        
            word_str = str(row['lemma'])
            value_str = str(row['frequency'])
            value_str = value_str + "\n"
            trimmed = word_str.lstrip()
            row_str = "\t".join([trimmed, value_str])
            f.write(row_str)
        
# Load dataframe to txt and csv at defined source file
load_N1grams_toCSV(lemma_freq_N1)
load_N1grams_toTXT(lemma_freq_N1)